In [1]:
import pandas as pd
import numpy as np

In [2]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.metrics import fbeta_score, make_scorer
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score
from mlxtend.classifier import StackingCVClassifier
from sklearn import model_selection

# Reading Data

In [3]:
df = pd.read_csv('../../Dataset/NSL_new.csv')

In [4]:
df = df.drop( df[ (df.label != "normal") & (df.label != "neptune") & (df.label != "back") & (df.label != "land") & (df.label != "pod") & (df.label != "smurf") & (df.label != "teardrop") & (df.label != "teardrop") & (df.label != "mailbomb") & (df.label != "apache2") & (df.label != "processtable") & (df.label != "udpstorm") & (df.label != "worm")].index )
print(df.shape)

(113270, 123)


In [5]:
#df = df.sample(n=20000)

In [6]:
df.head()

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,flag_RSTO,flag_RSTOS0,flag_RSTR,flag_S0,flag_S1,flag_S2,flag_S3,flag_SF,flag_SH,label
0,0,491,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,normal
1,0,146,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,normal
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,neptune
3,0,232,8153,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,normal
4,0,199,420,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,normal


In [7]:
X = df.drop(['label'], axis=1)
y = df['label']

In [8]:
scoring_metrics = {'accuracy' : make_scorer(accuracy_score), 
           'precision' : make_scorer(precision_score, average="micro"),
           'recall' : make_scorer(recall_score, average="micro"), 
           'f1_score' : make_scorer(f1_score, average="micro")}

In [9]:
y = y.replace( to_replace =  'normal' , value = 0 )

y = y.replace( to_replace =  'neptune' , value = 1 )
y = y.replace( to_replace =  'back' , value = 1 )
y = y.replace( to_replace =  'land' , value = 1 )
y = y.replace( to_replace =  'pod' , value = 1 )
y = y.replace( to_replace =  'smurf' , value = 1 )
y = y.replace( to_replace =  'teardrop' , value = 1 )
y = y.replace( to_replace =  'mailbomb' , value = 1 )
y = y.replace( to_replace =  'apache2' , value = 1 )
y = y.replace( to_replace =  'processtable' , value = 1 )
y = y.replace( to_replace =  'udpstorm' , value = 1 )
y = y.replace( to_replace =  'worm' , value = 1 )

# Chi Squared

In [10]:
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline
from sklearn.feature_selection import SelectKBest, chi2

In [11]:
print("before transform:",X)
selector=SelectKBest(score_func=chi2,k=3)
fit = selector.fit(X,y)
features = fit.transform(X)
print("scores_:",fit.scores_)
print("pvalues_:",fit.pvalues_)
print("selected index:",fit.get_support(True))
X = fit.transform(X)
print("after transform:", X)


before transform:         duration  src_bytes  dst_bytes  land  wrong_fragment  urgent  hot  \
0              0        491          0     0               0       0    0   
1              0        146          0     0               0       0    0   
2              0          0          0     0               0       0    0   
3              0        232       8153     0               0       0    0   
4              0        199        420     0               0       0    0   
...          ...        ...        ...   ...             ...     ...  ...   
125968         0          0          0     0               0       0    0   
125969         8        105        145     0               0       0    0   
125970         0       2231        384     0               0       0    0   
125971         0          0          0     0               0       0    0   
125972         0        151          0     0               0       0    0   

        num_failed_logins  logged_in  num_compromised  ..

# adaboost , Random Forest - mlp

In [12]:

from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
from vecstack import stacking



In [13]:
from sklearn.metrics import accuracy_score , precision_score , recall_score , f1_score


scoring_metrics = { accuracy_score , precision_score , recall_score , f1_score }

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [15]:
# Create Base Learners
base_learners = [  RandomForestClassifier(n_estimators=90, min_samples_split=0.1 , max_depth=19 ) ,
                    AdaBoostClassifier(learning_rate=0.1 , n_estimators=1000)      
                ]
meta_learner = MLPClassifier(alpha=0.01 , hidden_layer_sizes=20 , learning_rate='constant' , solver='adam')

In [16]:
# Initialize Stacking Classifier with the Meta Learner
S_train, S_test = stacking(base_learners,                   
                           X_train, y_train, X_test,   
                           regression=False, 
     
#                            mode='oof_pred_bag', 
       
                           needs_proba=False,
         
                           save_dir=None, 
            
                           metric = accuracy_score , 
    
                           n_folds=4, 
                 
                           stratified=True,
            
                           shuffle=True,  
            
                           random_state=0,    
         
                           verbose=2)

task:         [classification]
n_classes:    [2]
metric:       [accuracy_score]
mode:         [oof_pred_bag]
n_models:     [2]

model  0:     [RandomForestClassifier]
    fold  0:  [0.93241812]
    fold  1:  [0.93956917]
    fold  2:  [0.92650305]
    fold  3:  [0.93202084]
    ----
    MEAN:     [0.93262779] + [0.00463971]
    FULL:     [0.93262779]

model  1:     [AdaBoostClassifier]
    fold  0:  [0.96985080]
    fold  1:  [0.97064536]
    fold  2:  [0.96799682]
    fold  3:  [0.96963009]
    ----
    MEAN:     [0.96953077] + [0.00096276]
    FULL:     [0.96953077]



In [17]:
model = meta_learner.fit(S_train, y_train)


In [18]:
y_pred = meta_learner.predict(S_test)


In [19]:
print('Accuracy score: [%.8f]' % accuracy_score(y_test, y_pred))
print('Precision score: [%.8f]' % precision_score(y_test, y_pred))
print('recall score: [%.8f]' % recall_score(y_test, y_pred))

Accuracy score: [0.96896795]
Precision score: [0.93478482]
recall score: [0.99329730]


In [20]:
stacking1 = { 'accuracy' : accuracy_score(y_test, y_pred) , 'Precision' : precision_score(y_test, y_pred) , 'recall' : recall_score(y_test, y_pred) } 

In [21]:
stacking1 = pd.DataFrame(stacking1 , index=[0,1] )

In [22]:
print(stacking1)

   accuracy  Precision    recall
0  0.968968   0.934785  0.993297
1  0.968968   0.934785  0.993297


In [23]:
stacking1.to_csv('NSL_DOS_Stacking_adaboost-randomForest-mlp_ChiSquared.csv')

# adaboost , KNN - mlp

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [25]:
# Create Base Learners
base_learners = [KNeighborsClassifier(n_neighbors= 1) ,
                 AdaBoostClassifier(learning_rate=0.1 , n_estimators=1000)          
                ]
meta_learner = MLPClassifier(alpha=0.01 , hidden_layer_sizes=20 , learning_rate='constant' , solver='adam')

In [26]:
# Initialize Stacking Classifier with the Meta Learner
S_train, S_test = stacking(base_learners,                   
                           X_train, y_train, X_test,   
                           regression=False, 
     
#                            mode='oof_pred_bag', 
       
                           needs_proba=False,
         
                           save_dir=None, 
            
                           metric= accuracy_score , 
    
                           n_folds=4, 
                 
                           stratified=True,
            
                           shuffle=True,  
            
                           random_state=0,    
         
                           verbose=2)

task:         [classification]
n_classes:    [2]
metric:       [accuracy_score]
mode:         [oof_pred_bag]
n_models:     [2]

model  0:     [KNeighborsClassifier]
    fold  0:  [0.63534034]
    fold  1:  [0.97254348]
    fold  2:  [0.97055708]
    fold  3:  [0.97170478]
    ----
    MEAN:     [0.88753642] + [0.14560718]
    FULL:     [0.88753642]

model  1:     [AdaBoostClassifier]
    fold  0:  [0.96985080]
    fold  1:  [0.97064536]
    fold  2:  [0.96799682]
    fold  3:  [0.96963009]
    ----
    MEAN:     [0.96953077] + [0.00096276]
    FULL:     [0.96953077]



In [27]:
model = meta_learner.fit(S_train, y_train)

In [28]:
y_pred = meta_learner.predict(S_test)


In [29]:
stacking2 = { 'accuracy' : accuracy_score(y_test, y_pred) , 'Precision' : precision_score(y_test, y_pred) , 'recall' : recall_score(y_test, y_pred) } 

In [30]:
stacking2 = pd.DataFrame(stacking2 , index=[0,1] )
print(stacking2)

   accuracy  Precision    recall
0  0.971616   0.935181  0.999784
1  0.971616   0.935181  0.999784


In [31]:
stacking2.to_csv('NSL_DOS_Stacking_adaboost-KNN-mlp_ChiSquared.csv')

# adaboost , Descision Tree - mlp

In [32]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [33]:
from sklearn.tree import tree

# Create Base Learners
base_learners = [tree.DecisionTreeClassifier(max_depth= 13 , min_samples_split = 10 ) ,
                 AdaBoostClassifier(learning_rate=0.1 , n_estimators=1000)          
                ]
meta_learner = MLPClassifier(alpha=0.01 , hidden_layer_sizes=20 , learning_rate='constant' , solver='adam')

C:\Users\nayer\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.tree.tree module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.tree. Anything that cannot be imported from sklearn.tree is now part of the private API.
  warnings.warn(message, FutureWarning)


In [34]:
# Initialize Stacking Classifier with the Meta Learner
S_train, S_test = stacking(base_learners,                   
                           X_train, y_train, X_test,   
                           regression=False, 
     
#                            mode='oof_pred_bag', 
       
                           needs_proba=False,
         
                           save_dir=None, 
            
                           metric= accuracy_score , 
    
                           n_folds=4, 
                 
                           stratified=True,
            
                           shuffle=True,  
            
                           random_state=0,    
         
                           verbose=2)

task:         [classification]
n_classes:    [2]
metric:       [accuracy_score]
mode:         [oof_pred_bag]
n_models:     [2]

model  0:     [DecisionTreeClassifier]
    fold  0:  [0.97179306]
    fold  1:  [0.97236691]
    fold  2:  [0.97060122]
    fold  3:  [0.97166063]
    ----
    MEAN:     [0.97160546] + [0.00063768]
    FULL:     [0.97160546]

model  1:     [AdaBoostClassifier]
    fold  0:  [0.96985080]
    fold  1:  [0.97064536]
    fold  2:  [0.96799682]
    fold  3:  [0.96963009]
    ----
    MEAN:     [0.96953077] + [0.00096276]
    FULL:     [0.96953077]



In [35]:
model = meta_learner.fit(S_train, y_train)

In [36]:
y_pred = meta_learner.predict(S_test)


In [37]:
stacking3 = { 'accuracy' : accuracy_score(y_test, y_pred) , 'Precision' : precision_score(y_test, y_pred) , 'recall' : recall_score(y_test, y_pred) } 

In [38]:
stacking3 = pd.DataFrame(stacking3 , index=[0,1] )
print(stacking2)

   accuracy  Precision    recall
0  0.971616   0.935181  0.999784
1  0.971616   0.935181  0.999784


In [39]:
stacking3.to_csv('NSL_DOS_Stacking_adaboost-DecisionTree-mlp_ChiSquared.csv')

# adaboost , SVM - mlp